In [1]:
import cv2
import numpy as np

In [2]:
def draw_circle(event,x,y,flags,param):
    global count
    if event == cv2.EVENT_LBUTTONUP:
        circles2.append((x, y))
        positions.append([x,y])
        if(count!=3):
            positions2.append([x,y])
        elif(count==3):
            positions2.insert(2,[x,y])
        count+=1
        
def mouse_drawing_one(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        circles1.append((x, y))
        pos1.append([x, y])
        print(circles1)
        print(pos1)

def mouse_drawing_two(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        circles_roi.append((x, y))
        pos_roi.append([x, y])
        print(circles_roi)
        print(pos_roi)

In [3]:
guide = cv2.imread('guide.png')
cap = cv2.VideoCapture('guide.mp4')

In [4]:
positions=[] 
positions2=[]
count=0

pos1 = []
circles1 = []
circles2 = []

cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_circle)

cv2.namedWindow("Frame 1")
cv2.setMouseCallback("Frame 1", mouse_drawing_one)

while(True):
    cv2.imshow('image',guide)

    ret, frame = cap.read()
    h, w = frame.shape[:2]

    for center_position in circles1:
        cv2.circle(frame, center_position, 10, (255, 0, 0), -1)

    for center_position in circles2:
        cv2.circle(guide, center_position, 10, (255, 0, 0), -1)

    cv2.imshow("Frame 1", frame)

    k = cv2.waitKey(20) & 0xFF
    if k == 27:
        break
    elif k == ord("d"):
        positions=[] 
        positions2=[]
        count=0
        pos1 = []
        circles1 = []
        circles2 = []
cv2.destroyAllWindows()

[(476, 368)]
[[476, 368]]
[(476, 368), (806, 364)]
[[476, 368], [806, 364]]
[(476, 368), (806, 364), (278, 512)]
[[476, 368], [806, 364], [278, 512]]
[(476, 368), (806, 364), (278, 512), (979, 525)]
[[476, 368], [806, 364], [278, 512], [979, 525]]


In [5]:
circles_roi = []
pos_roi = []

cv2.namedWindow("ROI Select")
cv2.setMouseCallback("ROI Select", mouse_drawing_two)

while(True):

    ret, dp = cap.read()

    h1,w1 = dp.shape[:2]

    height, width = guide.shape[:2]

    pts1=np.float32([pos1[0], pos1[1], pos1[2], pos1[3]])
    pts2=np.float32(positions)


    h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC,5.0)

    height, width, channels = guide.shape
    im1Reg = cv2.warpPerspective(dp, h, (width, height))

    mask2 = np.zeros(guide.shape, dtype=np.uint8)

    # roi_corners2 = np.int32(positions2)
    # channel_count2 = building.shape[2]  
    # ignore_mask_color2 = (255,)*channel_count2
    # cv2.fillConvexPoly(mask2, roi_corners2, ignore_mask_color2)
    # mask2 = cv2.bitwise_not(mask2)
    # masked_image2 = cv2.bitwise_and(building, mask2)
    # final = cv2.bitwise_or(masked_image2, im1Reg)

    final = cv2.add(im1Reg, guide)

    # bldg_copy = building.copy()
    # bldg_copy[np.where(im1Reg)] = 0
    # subbed = cv2.subtract(bldg_copy, im1Reg)
    # final = cv2.add(im1Reg, subbed)
    
    for center_position in circles_roi:
        cv2.circle(final, center_position, 10, (255, 0, 0), -1)
    
    # cv2.imshow('mask',mask2)
    # cv2.imshow('subbed',subbed)
    # cv2.imshow('warped',im1Reg)

    if len(pos_roi) < 4:
        cv2.imshow('ROI Select',final)
    elif len(pos_roi) == 4:
        pts1 = np.float32([pos_roi[0], pos_roi[1], pos_roi[2], pos_roi[3]])
        pts2 = np.float32([[0, 0], [1280, 0], [0, 720], [1280, 720]])
        matrix = cv2.getPerspectiveTransform(pts1, pts2)
        result = cv2.warpPerspective(final, matrix, (1280, 720))
        cv2.destroyWindow('ROI Select')
        cv2.imshow("ROI", result)

    k = cv2.waitKey(20) & 0xFF
    if k == 27:
        break
    elif k == ord("d"):
        circles_roi = []
        pos_roi = []

cv2.destroyAllWindows()

[(196, 89)]
[[196, 89]]
[(196, 89), (1127, 84)]
[[196, 89], [1127, 84]]
[(196, 89), (1127, 84), (157, 625)]
[[196, 89], [1127, 84], [157, 625]]
[(196, 89), (1127, 84), (157, 625), (1160, 646)]
[[196, 89], [1127, 84], [157, 625], [1160, 646]]
